In [1]:
import pandas as pd
import mlflow
from mlflow.models import infer_signature

mlflow.set_experiment("real_estate_price_prediction")
dataset = pd.read_csv("Real estate.csv")
dataset.drop(columns=["X5 latitude","X6 longitude"], inplace=True)
dataset.describe()

2025/12/18 19:57:14 INFO mlflow.tracking.fluent: Experiment with name 'real_estate_price_prediction' does not exist. Creating a new experiment.


,No,X1 transaction date,X2 house age,X3 distance to the nearest MRT station,X4 number of convenience stores,Y house price of unit area
count,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000
mean,207.500000,2013.148971,17.712560,1083.885689,4.094203,37.980193
std,119.655756,0.281967,11.392485,1262.109595,2.945562,13.606488
min,1.000000,2012.667000,0.000000,23.382840,0.000000,7.600000
25%,104.250000,2012.917000,9.025000,289.324800,1.000000,27.700000
50%,207.500000,2013.167000,16.100000,492.231300,4.000000,38.450000
75%,310.750000,2013.417000,28.150000,1454.279000,6.000000,46.600000
max,414.000000,2013.583000,43.800000,6488.021000,10.000000,117.500000


In [2]:
dataset.isnull().sum()

No                                        0
X1 transaction date                       0
X2 house age                              0
X3 distance to the nearest MRT station    0
X4 number of convenience stores           0
Y house price of unit area                0
dtype: int64

In [3]:
y=dataset["Y house price of unit area"]
X=dataset.drop(columns=["Y house price of unit area"])
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures


In [4]:
for i in range(1,11):
    with mlflow.start_run(run_name=f"polynomial_degree_{i}"):
        poly_features = PolynomialFeatures(degree=i)
        X_poly = poly_features.fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.18, random_state=42)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        mlflow.log_param("degree", i)
        mlflow.log_metric("mean_absolute_error", mae)
        mlflow.log_metric("mean_squared_error", mse)
        mlflow.log_metric("r2_score", r2)
        signature = infer_signature(X_train, model.predict(X_train))
        model_info = mlflow.sklearn.log_model(sk_model=model, artifact_path="poly_reg", signature=signature, input_example=X_train, registered_model_name=f"PolyRegModelDegree{i}")
        print(f"Mean Absolute Error: {mae} {'for degree ' + str(i)}")
        print(f"Mean Squared Error: {mse} {'for degree ' + str(i)}")
        print(f"R^2 Score: {r2} {'for degree ' + str(i)}")

/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
2025/12/18 19:57:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_bas

Mean Absolute Error: 5.63691137375381 for degree 1
Mean Squared Error: 59.70977358874787 for degree 1
R^2 Score: 0.6659804333725291 for degree 1


/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
Successfully registered model 'PolyRegModelDegree2'.
Created version '1' of model 'PolyRegModelDegree2'.
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.

Mean Absolute Error: 5.113820679426193 for degree 2
Mean Squared Error: 50.003236370891635 for degree 2
R^2 Score: 0.7202793054012908 for degree 2


/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
Successfully registered model 'PolyRegModelDegree3'.
Created version '1' of model 'PolyRegModelDegree3'.
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.

...

/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
Successfully registered model 'PolyRegModelDegree8'.
Created version '1' of model 'PolyRegModelDegree8'.
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:312: RuntimeWarning: divide by zero encountered in matmul
  intercept_ = y_offset - X_offset @ coef_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/

Mean Absolute Error: 1340.5912963104818 for degree 8
Mean Squared Error: 114786773.79056232 for degree 8
R^2 Score: -642122.1589348115 for degree 8


/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
Successfully registered model 'PolyRegModelDegree9'.
Created version '1' of model 'PolyRegModelDegree9'.
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:312: RuntimeWarning: divide by zero encountered in matmul
  intercept_ = y_offset - X_offset @ coef_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/

Mean Absolute Error: 7636.527714158013 for degree 9
Mean Squared Error: 4203673608.7064004 for degree 9
R^2 Score: -23515566.931881234 for degree 9
Mean Absolute Error: 22411.480706313134 for degree 10
Mean Squared Error: 19181009922.326843 for degree 10
R^2 Score: -107299562.1479028 for degree 10


/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_
Successfully registered model 'PolyRegModelDegree10'.
Created version '1' of model 'PolyRegModelDegree10'.


In [ ]:
poly_features = PolynomialFeatures(degree=3)
X_poly = poly_features.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.18, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Absolute Error: 5.1350540800541635
Mean Squared Error: 48.71757197729269
R^2 Score: 0.7274713786209289


/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/keshavsrisai/Documents/Pyproject/mlenv/lib/python3.13/site-packages/sklearn/linear_model/_base.py:280: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


: 

: 

In [6]:
print(model_info.model_uri)
print(model_info.run_id)


models:/m-cbfac9fc76104ae5b477edefa5a52935
44f7f5a9a7454647ab060544d4eb7231


In [8]:
from mlflow import sklearn

model_name="PolyRegModelDegree3"
model_v="latest"
uri = f"models:/{model_name}/{model_v}"

model = sklearn.load_model(uri)
model

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False
